In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

In [2]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a michelin star chef and expert in culinary arts. You know everything about famous dishes"},
  {"role": "user", "content": "What are the steps to make Beef Wellington."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "This is the most accurate recipe for Beef Wellington: https://www.gordonramsay.com/gr/recipes/beef-wellington"
          }
      }
  ]
}

@traceable(
  run_type="llm",
  metadata={
    "ls_provider": "groq", 
    "ls_model_name": "openai/gpt-oss-20b"
  }
)
def chat_model(messages: list):
  return output


chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'This is the most accurate recipe for Beef Wellington: https://www.gordonramsay.com/gr/recipes/beef-wellington'}}]}

In [3]:
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    reduce_fn=_reduce_chunks,
    metadata={
        "ls_provider": "my_provider", 
        "ls_model_name": "my_model"
    }
)
def my_streaming_chat_model(messages: list):
    for chunk in ["Hallo, " + messages[1]["content"] + "! How are you doing mate?"]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are an Australian bartender who likes to joke around."},
            {"role": "user", "content": "LazarBeam"},
        ],
    )
)

[{'choices': [{'message': {'content': 'Hallo, LazarBeam! How are you doing mate?',
     'role': 'assistant'}}]}]

In [ ]:
from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Drinks",
          "metadata": {"Bar": "The Baxter Inn"}
      }
      for r in results
  ]

@traceable(run_type="retriever")
def retrieve_docs(query):
  contents = [
    "Macallan 18", 
    "Negroni", 
    "Espresso Martini",
    "Sex on the Beach",
    "BroCode",
    "Smirnoff",
    "Patiala Peg"
  ]
  return _convert_docs(contents)

retrieve_docs("Searching for available drinks at the bar")

[{'page_content': 'Macallan 18',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}},
 {'page_content': 'Negroni',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}},
 {'page_content': 'Espresso Martini',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}},
 {'page_content': 'Sex on the Beach',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}},
 {'page_content': 'BroCode',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}},
 {'page_content': 'Smirnoff',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}},
 {'page_content': 'Patiala Peg',
  'type': 'Drinks',
  'metadata': {'Bar': 'The Baxter Inn'}}]

In [8]:
from langsmith import traceable
from groq import Groq
from typing import List, Optional
import json

groq_client = Groq()

@traceable(run_type="tool")
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17

@traceable(run_type="llm")
def call_groq(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
  return groq_client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=messages,
    temperature=0,
    tools=tools
  )

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
  response = call_groq(inputs, tools)
  tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
  location = tool_call_args["location"]
  unit = tool_call_args["unit"]
  tool_response_message = {
    "role": "tool",
    "content": json.dumps({
        "location": location,
        "unit": unit,
        "temperature": get_current_temperature(location, unit),
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
  }
  inputs.append(response.choices[0].message)
  inputs.append(tool_response_message)
  output = call_groq(inputs, None)
  return output

tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Asking Reynold Wolf for the current weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, eg. Charrlotte, NC"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "If the user is American, use Fahrenheit, otherwise use Celsius. Rest of the world is smart like that."
            }
          },
          "required": ["location", "unit"]
        }
      }
    }
]
inputs = [
  {"role": "system", "content": "You are a weather predicting God who can predict the world everywhere in the world."},
  {"role": "user", "content": "What is the weather today in Alaska?"},
]

ask_about_the_weather(inputs, tools)

ChatCompletion(id='chatcmpl-cf1821cc-d83b-43cb-92d1-264b08f28547', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in Anchorage, Alaska is **65\u202f°F**.  \n(Note: Alaska spans a huge area, so temperatures can vary widely across the state. If you need conditions for a different city or a more detailed forecast, let me know!', role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1759576516, model='openai/gpt-oss-20b', object='chat.completion', system_fingerprint='fp_c5a89987dc', usage=CompletionUsage(completion_tokens=54, prompt_tokens=856, total_tokens=910, completion_time=0.056807848, prompt_time=0.045830511, queue_time=0.042299062, total_time=0.102638359), usage_breakdown=None, x_groq={'id': 'req_01k6qexsydej8sepm7v90vtbsp'}, service_tier='on_demand')

In [11]:
from langsmith import traceable
import random


@traceable(run_type="tool")
def chess_courtship(battlefront: str):
    """Survey chess battlefields with counsel from medieval strategists"""
    
    battle_conditions = {
        "aggressive": "Knights charge and pawns advance in a bold gambit",
        "defensive": "Stonewall ranks hold fast behind sturdy bastions",
        "tactical": "Forks, pins, and skewers spring like ambushes from hedgerows",
        "positional": "Files and outposts are secured as lords fortify their keeps",
        "endgame": "Kings muster escorts as passed pawns march toward coronation"
    }
    stance = random.choice(list(battle_conditions.keys()))
    
    return {
        "battlefront": battlefront,
        "battle_brief": battle_conditions[stance],
        "pressure_gauge": random.randint(1, 10)
    }

@traceable(run_type="chain")
def marshal_chess_campaign():
    """Campaign plans across the medieval chess kingdoms"""
    battlefields = [
        "The Queenside Castle Ward",
        "The Sicilian Borderlands",
        "The Aether Ranks and Floating Files",
        "The Dragon Gate of the King’s March",
        "The Nimzo Manor and Whispering Lanes"
    ]
    
    dispatches = []
    for field in battlefields:
        report = chess_courtship(field)
        dispatches.append(
            f"In {report['battlefront']}: {report['battle_brief']} (Intensity: {report['pressure_gauge']}/10)"
        )
    
    return {
        "declaration": "Hear ye! The Royal Chess Council has declared its stratagems...",
        "front_reports": dispatches,
        "field_guidance": "Guard the center, claim open files, and let minor pieces seize noble outposts."
    }


royal_bulletin = marshal_chess_campaign()
print(royal_bulletin)


{'declaration': 'Hear ye! The Royal Chess Council has declared its stratagems...', 'front_reports': ['In The Queenside Castle Ward: Knights charge and pawns advance in a bold gambit (Intensity: 1/10)', 'In The Sicilian Borderlands: Files and outposts are secured as lords fortify their keeps (Intensity: 9/10)', 'In The Aether Ranks and Floating Files: Knights charge and pawns advance in a bold gambit (Intensity: 9/10)', 'In The Dragon Gate of the King’s March: Knights charge and pawns advance in a bold gambit (Intensity: 3/10)', 'In The Nimzo Manor and Whispering Lanes: Kings muster escorts as passed pawns march toward coronation (Intensity: 7/10)'], 'field_guidance': 'Guard the center, claim open files, and let minor pieces seize noble outposts.'}
